In [1]:
#list files in fast folder, get unique prefix, and in python call the script with the different parameters 
import os
import shlex

sampleID=[]
for subdir, dirs, files in os.walk('/Users/joelkim/Desktop/FAST'):
    for file in files:
      x=file[:16]
      if x not in sampleID:    
        sampleID.append(x)
      
print(sampleID)
len(sampleID)


      
  


['000008508669_E03', '000008508669_E02', '000008508669_G02', '000008508669_G03', '000008508669_A06', '000008508669_A07', '000008508669_C07', '000008508669_C06', '000008508669_H03', '000008508669_H02', '000008508669_B05', '000008508669_B04', '000008508669_E04', '000008508669_E05', '000008508669_G05', '000008508669_G04', '000008508669_B02', '000008508669_B03', '000008508669_D06', '000008508669_H04', '000008508669_F06', '000008508669_H05', '000008508669_D05', '000008508669_D04', '000008508669_F04', '000008508669_F05', '000008508669_A03', '000008508669_C03', '000008508669_C02', '000008508669_E06', '000008508669_G06', '000008508669_D02', '000008508669_D03', '000008508669_F03', '000008508669_H01', '000008508669_F02', '000008508669_B06', '000008508669_B07', '000008508669_A05', '000008508669_A04', '000008508669_C04', '000008508669_C05']


42

In [2]:
for sid in sampleID:
   subprocess.call(shlex.split('/Users/joelkim/Desktop/data/alignment.sh '+sid))





In [28]:
import os
import pandas as pd
import glob 
import numpy as np
import seaborn as sns
import scipy.stats as ss

os.chdir('/Users/joelkim/Desktop/data/aligned')
all_files = glob.glob("*ReadsPerGene.out.tab")

# Initialize dataframe list
df_list = []

# Loop through files
for file in all_files:
    
    # Read data file
    df = pd.read_table(file, header=None)

    # Drop the unwanted columns
    df.drop([2,3], axis=1, inplace=True)

    # Drop unwanted rows
    df.drop([0,1,2,3], axis=0, inplace=True)

    # Rename columns
    df.rename(columns={0: 'ensembl_gene_id', 1:'counts'}, inplace=True)

    # Extract sample label
    sample_label = file.split('_')[1].replace('ReadsPerGene.out.tab', '')

    # Add the sample label as a column to the dataframe
    df['sample_label'] = sample_label
    
    # Append dataframe
    df_list.append(df)

# Concatenate results
concatenated_dataframe = pd.concat(df_list)
concatenated_dataframe.head()


,ensembl_gene_id,counts,sample_label
4,ENSMUSG00000102693,0,G02
5,ENSMUSG00000064842,0,G02
6,ENSMUSG00000051951,25,G02
7,ENSMUSG00000102851,0,G02
8,ENSMUSG00000103377,0,G02


In [2]:
count_dataframe = concatenated_dataframe.pivot(index='ensembl_gene_id', columns='sample_label', values='counts')
count_dataframe.head()


sample_label,A03,A04,A05,A06,A07,B02,B03,B04,B05,B06,...,G02,G03,G04,G05,G06,H01,H02,H03,H04,H05
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSMUSG00000000001,1167,1176,396,690,988,1458,1288,1588,1272,1122,...,881,1334,465,1835,1808,519,1731,909,1608,1465
ENSMUSG00000000003,0,0,0,0,0,0,0,1,1,0,...,0,2,0,1,0,0,0,0,0,0
ENSMUSG00000000028,39,9,6,29,9,18,66,28,32,48,...,33,24,7,38,46,63,70,19,13,46
ENSMUSG00000000031,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,24,0,0,0,0
ENSMUSG00000000037,2,4,4,5,3,2,0,1,9,0,...,3,5,1,1,0,2,0,0,1,7


In [3]:
#load HGNC data
MGI_dataframe = pd.read_table('/Users/joelkim/Desktop/ensembl_mousegenes.txt', index_col="Gene stable ID")

#filter protein coding genes
MGI_dataframe = MGI_dataframe[MGI_dataframe["Gene type"]=="protein_coding"]
MGI_dataframe.head()




,Gene type,MGI symbol
Gene stable ID,,
ENSMUSG00000064341,protein_coding,mt-Nd1
ENSMUSG00000064345,protein_coding,mt-Nd2
ENSMUSG00000064351,protein_coding,mt-Co1
ENSMUSG00000064354,protein_coding,mt-Co2
ENSMUSG00000064356,protein_coding,mt-Atp8


In [4]:
#Map gene symbols
result = pd.merge(count_dataframe, MGI_dataframe, left_index=True, right_index=True, how='inner')

# Drop gene type column
result.drop("Gene type", axis=1, inplace=True)

# Rename columns
result.rename(columns={0: 'ensembl_gene_id', 1:'counts'}, inplace=True)

# Aggregate genes with same MGI symbol
result = result.groupby('MGI symbol').sum()

print(result.shape)
result.head()

(21898, 42)


,A03,A04,A05,A06,A07,B02,B03,B04,B05,B06,...,G02,G03,G04,G05,G06,H01,H02,H03,H04,H05
MGI symbol,,,,,,,,,,,,,,,,,,,,,
0610009B22Rik,50,69,10,38,30,71,112,81,74,59,...,53,69,16,81,78,16,110,52,85,84
0610009O20Rik,52,55,4,60,59,77,75,46,86,66,...,75,89,24,79,86,44,88,38,93,109
0610010F05Rik,155,143,42,148,94,202,146,214,150,151,...,188,205,95,291,308,32,213,135,228,238
0610010K14Rik,6,4,6,3,4,12,8,5,7,2,...,26,8,3,13,10,1,3,9,9,4
0610012G03Rik,106,131,38,125,62,164,224,195,121,132,...,89,105,50,251,242,34,180,73,245,224


In [5]:
result = result.transpose()
result.head()

MGI symbol,0610009B22Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610040J01Rik,1110002E22Rik,1110004E09Rik,...,mt-Co2,mt-Co3,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd4l,mt-Nd5,mt-Nd6
A03,50,52,155,6,106,4,117,3,3,30,...,94,139,15063,27361,2665,129,6403,0,7201,223
A04,69,55,143,4,131,3,129,125,0,23,...,58,111,6744,12064,1863,58,3400,0,5097,213
A05,10,4,42,6,38,0,47,27,0,5,...,15,27,1758,3391,562,15,1003,0,1539,42
A06,38,60,148,3,125,2,80,81,1,29,...,52,129,7157,17470,1604,57,3332,0,5172,224
A07,30,59,94,4,62,6,102,59,0,8,...,23,55,4305,8063,1430,24,2019,0,2839,120


In [7]:
#Select out Alveolar Macrophages 
#alveolar_macrophage = result.loc[['A03', 'B03', 'C06', 'D06', 'E04', 'E06']]
#alveolar_macrophage.head()

#select out inflmmatory monocytes from lung
#inflammatory_monocytes_lung = result.loc[['B02', 'C02', 'D02', 'A04', 'B04', 'E05', 'F05', 'G05']] 
#print(inflammatory_monocytes_lung)

#select out resident monocytes from lung
#resident_monocytes_lung = result.loc[['E02', 'F02', 'G02', 'C04', 'D04', 'H04', 'A06', 'B06']] 
#print(resident_monocytes_lung)

#select out inflmmatory monocytes from blood
#inflammatory_monocytes_blood = result.loc[['C03', 'D03', 'E03', 'G04', 'H03', 'A05', 'G06', 'H05']] 
#print(inflammatory_monocytes_blood)

#select out resident monocytes from blood
#resident_monocytes_blood = result.loc[['F03', 'G03', 'H02', 'B05', 'C05', 'D05', 'A07', 'B07', 'C07']] 
#print(resident_monocytes_blood)

#Select out inflammatory monocytes 
inflammatory_monocytes = result.loc[['C03', 'D03', 'E03', 'G04', 'H03', 'A05', 'G06', 'H05', 'B02', 'C02', 'D02', 'A04', 'B04', 'E05', 'F05', 'G05']] 
print(inflammatory_monocytes)

#Select out resident monocytes
#resident_monocytes = result.loc[['F03', 'G03', 'H02', 'B05', 'C05', 'D05', 'E02', 'F02', 'G02', 'C04', 'D04', 'H04', 'A06', 'B06', 'A07', 'B07', 'C07']] 
#resident_monocytes.head()


MGI symbol  0610009B22Rik  0610009O20Rik  0610010F05Rik  0610010K14Rik  \
F03                   116             88            146             19   
G03                    69             89            205              8   
H02                   110             88            213              3   
B05                    74             86            150              7   
C05                    44             82            175              7   
D05                    42             50             70             10   
A07                    30             59             94              4   
B07                    57             66            205              8   
C07                    42             55            177             12   

MGI symbol  0610012G03Rik  0610030E20Rik  0610037L13Rik  0610040J01Rik  \
F03                   215              6            392            144   
G03                   105              4            258             75   
H02                   180            

In [8]:
#load xyz
xyz = pd.read_table('/Users/joelkim/Desktop/xyz.txt', index_col=0)
xyz.head()


,Sample Name
Well #,
F03,Mo.Ly6C-.Bl#1
G03,Mo.Ly6C-.Bl#2
H02,Mo.Ly6C-.Bl#3
B05,Mo.Ly6C-.Bl.KPd17#1
C05,Mo.Ly6C-.Bl.KPd17#2


In [9]:
#merge
result = pd.concat([resident_monocytes_blood, xyz], axis=1)
result.head()

,0610009B22Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610040J01Rik,1110002E22Rik,1110004E09Rik,...,mt-Co3,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd4l,mt-Nd5,mt-Nd6,Sample Name
Well #,,,,,,,,,,,,,,,,,,,,,
F03,116,88,146,19,215,6,392,144,5,46,...,159,12020,23061,3210,110,4382,0,6466,197,Mo.Ly6C-.Bl#1
G03,69,89,205,8,105,4,258,75,14,13,...,53,4964,10378,1701,45,2188,0,3286,140,Mo.Ly6C-.Bl#2
H02,110,88,213,3,180,3,207,0,4,50,...,255,19067,31830,3424,188,7734,0,9676,340,Mo.Ly6C-.Bl#3
B05,74,86,150,7,121,13,197,48,6,15,...,71,5945,13208,1969,62,2525,0,4060,164,Mo.Ly6C-.Bl.KPd17#1
C05,44,82,175,7,88,8,205,58,1,25,...,58,6202,11667,2083,87,2658,0,3996,191,Mo.Ly6C-.Bl.KPd17#2


In [10]:
result.set_index('Sample Name')

,0610009B22Rik,0610009O20Rik,0610010F05Rik,0610010K14Rik,0610012G03Rik,0610030E20Rik,0610037L13Rik,0610040J01Rik,1110002E22Rik,1110004E09Rik,...,mt-Co2,mt-Co3,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd3,mt-Nd4,mt-Nd4l,mt-Nd5,mt-Nd6
Sample Name,,,,,,,,,,,,,,,,,,,,,
Mo.Ly6C-.Bl#1,116,88,146,19,215,6,392,144,5,46,...,65,159,12020,23061,3210,110,4382,0,6466,197
Mo.Ly6C-.Bl#2,69,89,205,8,105,4,258,75,14,13,...,47,53,4964,10378,1701,45,2188,0,3286,140
Mo.Ly6C-.Bl#3,110,88,213,3,180,3,207,0,4,50,...,89,255,19067,31830,3424,188,7734,0,9676,340
Mo.Ly6C-.Bl.KPd17#1,74,86,150,7,121,13,197,48,6,15,...,44,71,5945,13208,1969,62,2525,0,4060,164
Mo.Ly6C-.Bl.KPd17#2,44,82,175,7,88,8,205,58,1,25,...,52,58,6202,11667,2083,87,2658,0,3996,191
Mo.Ly6C-.Bl.KPd17#3,42,50,70,10,50,18,195,45,0,13,...,32,57,4017,8427,1537,35,1698,0,2914,88
Mo.Ly6C-Bl.KPd28#1,30,59,94,4,62,6,102,59,0,8,...,23,55,4305,8063,1430,24,2019,0,2839,120
Mo.Ly6C-Bl.KPd28#2,57,66,205,8,131,6,156,114,5,20,...,46,75,5489,12133,1575,39,2666,0,4107,185
Mo.Ly6C-Bl.KPd28#3,42,55,177,12,91,7,192,99,5,18,...,39,75,5569,10119,1528,63,2575,0,3735,188


In [30]:
df = result.set_index('Sample Name').transpose()
df.head()

Sample Name,Mo.Ly6C-.Bl#1,Mo.Ly6C-.Bl#2,Mo.Ly6C-.Bl#3,Mo.Ly6C-.Bl.KPd17#1,Mo.Ly6C-.Bl.KPd17#2,Mo.Ly6C-.Bl.KPd17#3,Mo.Ly6C-Bl.KPd28#1,Mo.Ly6C-Bl.KPd28#2,Mo.Ly6C-Bl.KPd28#3
0610009B22Rik,116,69,110,74,44,42,30,57,42
0610009O20Rik,88,89,88,86,82,50,59,66,55
0610010F05Rik,146,205,213,150,175,70,94,205,177
0610010K14Rik,19,8,3,7,7,10,4,8,12
0610012G03Rik,215,105,180,121,88,50,62,131,91


In [31]:
#normalize
library_sizes = df.sum()
print(library_sizes)

Sample Name
Mo.Ly6C-.Bl#1          6276926
Mo.Ly6C-.Bl#2          4336518
Mo.Ly6C-.Bl#3          4772099
Mo.Ly6C-.Bl.KPd17#1    4494657
Mo.Ly6C-.Bl.KPd17#2    4319744
Mo.Ly6C-.Bl.KPd17#3    3152558
Mo.Ly6C-Bl.KPd28#1     3003171
Mo.Ly6C-Bl.KPd28#2     3874316
Mo.Ly6C-Bl.KPd28#3     4225217
dtype: int64


In [32]:
col_normalized_dataframe = df/library_sizes
log_normalized_dataframe = np.log10(col_normalized_dataframe+1)
zscore_dataframe = log_normalized_dataframe.apply(ss.zscore())

TypeError: zscore() missing 1 required positional argument: 'a'

4.0

In [ ]:
result.set_index('Sample Name').transpose().to_csv("/Users/joelkim/Desktop/resident_monocytes_blood.txt", index=True, sep="\t")



In [907]:
#load Microglia Data
Microglia_df = pd.read_table('/Users/joelkim/Desktop/microglia_dataframe.txt', index_col=0, header=1)

# Drop unwanted rows
Microglia_dataframe = Microglia_df.iloc[27:]
Microglia_dataframe.head()
#drop(df.index[:3], inplace=True)
#Add Microglia Data
#result = pd.merge(result, Microglia_dataframe, left_index=True, right_index=True, how='inner')

#print(result.shape)
#result.head()

,WT1,WT2,WT3,WT4,WT5,WT6,AD1neg,AD2neg,AD3neg,AD4neg,AD5neg,AD6neg,AD1pos,AD2pos,AD3pos,AD4pos,AD5pos,AD6pos
tracking_id,,,,,,,,,,,,,,,,,,
0610005C13Rik,0.154913,0.359832,0.313995,0.0000,0.00000,0.304762,0.00000,0.159557,0.00000,0.112995,0.468237,0.00000,0.00000,0.143404,0.115422,0.00000,0.00000,0.00000
0610007P14Rik,30.776200,26.747800,27.489800,25.9106,34.12340,35.143600,35.03730,29.294800,45.32030,35.983400,28.316000,22.92880,50.08520,48.321200,41.393500,46.38860,41.87740,48.53060
0610009B22Rik,55.675500,49.309700,43.094800,29.2697,41.96480,42.222800,42.78470,43.922100,41.51480,59.653900,43.606400,37.83700,71.90940,37.352000,37.818400,62.92990,42.86020,34.41160
0610009L18Rik,6.458740,2.659970,3.999430,2.7999,1.65649,2.812070,3.83268,1.718360,9.73787,1.856480,1.926020,4.20296,2.12774,1.760870,18.246400,3.57263,2.49934,8.09917
0610009O20Rik,16.928500,23.628900,15.859400,17.4186,16.37220,12.904000,18.00670,17.046600,17.95750,17.717100,19.044800,16.61080,13.87590,9.875300,12.428000,10.59620,19.06050,14.60470


In [67]:
result.to_csv("/Users/joelkim/Desktop/mouse_lungcancer.txt", index=True, sep="\t")
#result.to_csv("/Users/joelkim/Desktop/microglia_lungcancer.txt", index=True, sep="\t")
#Microglia_dataframe.to_csv("/Users/joelkim/Desktop/microglia.txt", index=True, sep="\t")
#alveolar_macrophage.to_csv("/Users/joelkim/Desktop/alveolar_macrophage.txt", index=True, sep="\t")



In [ ]:
#create PCA plot
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(result)
pca.explained_variance_ratio_
#plt.figure()

#plt.show()
#plt_pca(pca, MGI_dataframe, color_by='condition')